In [ ]:
from tkinter import *
from pynput.mouse import Button, Controller

In [ ]:
m = Controller()

In [ ]:
def hover(xy):
    """ Move mouse to xy position """
    m.position = xy 

In [ ]:
def lclick(xy):
    """ Left click """
    hover(xy)
    m.click(Button.left, count=1)

In [ ]:
def rclick(xy):
    """ Right click """
    hover(xy)
    m.click(Button.right, count=1)

In [ ]:
def dclick(xy):
    """ Double click """
    hover(xy)
    m.click(Button.left, count=2)

In [ ]:
def mpos():
    """ Current mouse position """
    return m.position

In [ ]:
def screen_size():
    """ Get screen resolution using tkinter (on linux,mac tkinter needs to be installed) """
    root = Tk()
    width = root.winfo_screenwidth()
    height = root.winfo_screenheight()
    
    return width, height

In [ ]:
from mss import mss

# The simplest use, save a screen shot of the 1st monitor
with mss() as sct:
    sct.shot()

In [ ]:
# http://effbot.org/tkinterbook/canvas.htm#Tkinter.Canvas.create_rectangle-method

In [2]:
import os
from tkinter import Tk, Canvas, PhotoImage, YES, BOTH, NW, Toplevel
from mss import mss, tools


def full_screenshot(fn="img"):

    # Primary monitor screenshot

    filename = os.path.relpath(f'{fn}.png')

    with mss() as sct:
        sct.shot(mon=1, output=filename)

    return filename


def area_screenshot(sx, sy, ex, ey, fn="img"):

    try:
        os.mkdir("snaps")
    except:
        pass

    with mss() as sct:
        # The screen part to capture
        monitor = {"top": sy, "left": sx, "width": ex-sx, "height": ey-sy, "mon":1}
        filename = os.path.relpath("./snaps/" + fn + "-{top}x{left}_{width}x{height}.png".format(**monitor))

        # Grab the data
        sct_img = sct.grab(monitor)

        # Save to the picture file
        tools.to_png(sct_img.rgb, sct_img.size, output=filename)

        del sct


    return filename



class _Snap:
    
    """
        Use Tkinter and Canvas to make a full primary monitor screenshot
        Draw an rectangle on the image background and on left click release 
        Take a screenshot of what is inside that rectangle
        sxy - start
        exy - end
        cxy - current
    """
    
    def __init__(self, name="snap"):
        
        self.name = name
        self.snap = None
    
        self.window = Tk()
        #self.window = Toplevel(root)
        
        self.window.title("Snip picture")
        self.window.attributes('-fullscreen', True)  
        
        self.c = Canvas(self.window, cursor="cross")
        self.c.pack(expand=YES, fill=BOTH)

        fname = full_screenshot("temp")

        bg = PhotoImage(file=fname)
        self.c.create_image(0, 0, image=bg, anchor=NW)

        self.window.bind("<Escape>", self.quit_fullscreen)
        self.c.bind("<ButtonPress-1>", self.on_press)
        self.c.bind("<B1-Motion>", self.on_drag)
        self.c.bind("<ButtonRelease-1>", self.on_release)

        self.window.mainloop()
        #root.mainloop()

        
    def quit_fullscreen(self, event):
        self.window.attributes("-fullscreen", False)
                
    def clear_canvas(self):
        
        self.c.delete(self.rect)
        self.c.delete(self.d1)
        self.c.delete(self.d2)
        
        self.window.update()
    
        
    def on_press(self, event):
        #print("Pressed", event)
        
        self.sx = event.x
        self.sy = event.y
        
        self.rect = self.c.create_rectangle(self.sx, self.sy, self.sx, self.sy, width=2, fill="", dash=(3,5), outline="green3")
        
        self.d1 = self.c.create_line(self.sx, self.sy, self.sx, self.sy, width=1.2, fill="green2")
        self.d2 = self.c.create_line(self.sx, self.sy, self.sx, self.sy, width=1.2, fill="green2")
        
        
    def on_drag(self, event):
        #print("Draged", event)
        self.cx = event.x
        self.cy = event.y

        self.c.coords(self.rect, self.sx, self.sy, self.cx, self.cy)
        self.c.coords(self.d1, self.sx, self.sy, self.cx, self.cy)
        self.c.coords(self.d2, self.cx, self.sy, self.sx, self.cy)
        
        
    def on_release(self, event):
        #print("Released", event)
        self.ex = event.x
        self.ey = event.y
        
        self.clear_canvas()
        self.snap = area_screenshot(self.sx, self.sy, self.ex, self.ey, self.name)
        self.close_snip_window()
        
        
    def close_snip_window(self):
        os.remove("temp.png")
        self.window.withdraw()
        self.window.quit()
        
        
        
    
#Main function
def take_snap():
    """
        "sxy": xy start coordinates,
        "exy": xy end coordinates,
        "cxy": center of area image,
        "snap": file path of area screenshoted
    """
    
    obj = _Snap()
    
    cx = (obj.sx + obj.ex) / 2
    cy = (obj.sy + obj.ey) / 2
    
    data = {
        "sxy": (obj.sx, obj.sy,),
        "exy": (obj.ex, obj.ey,),
        "cxy": (cx, cy,),
        "snap": obj.snap
    }
    
    del obj

    return data



take_snap()


TclError: image "pyimage2" doesn't exist

In [ ]:
# TOO LINKED MUST BE DECOUPELD

import os
from tkinter import *
from mss import mss, tools


class Snaper:
        
    def __init__(self):
        self.window = Tk()
        self.window.title("Snip picture")
        self.window.attributes('-fullscreen', True)  
        self.fullScreenState = False
        
        self.c = Canvas(self.window, cursor="cross")
        self.c.pack(expand=YES, fill=BOTH)
        
        self.filename = self.full_screenshot()
        
        bg = PhotoImage(file=self.filename)
        self.c.create_image(0, 0, image=bg, anchor=NW)
        
        
        self.window.bind("<Escape>", self.quitFullScreen)
        self.c.bind("<ButtonPress-1>", self.on_press)
        self.c.bind("<B1-Motion>", self.on_drag)
        self.c.bind("<ButtonRelease-1>", self.on_release)
        

        self.window.mainloop()
        
        
        
    def full_screenshot(self):
        # Primary monitor screenshot
        with mss() as sct:
            filename = sct.shot(output=os.path.relpath('temp.png'))
            
        return filename
    
    
    def area_screenshot(self):
        
        with mss() as sct:
            # The screen part to capture
            monitor = {"top": self.sy, "left": self.sx, "width": self.ex-self.sx, "height": self.ey-self.sy}
            self.output = "snap-{top}x{left}_{width}x{height}.png".format(**monitor)

            # Grab the data
            sct_img = sct.grab(monitor)

            # Save to the picture file
            tools.to_png(sct_img.rgb, sct_img.size, output=self.output)
            
            
    def on_press(self, event):
        #print("Pressed", event)
        
        self.sx = event.x
        self.sy = event.y
        
        self.rect = self.c.create_rectangle(self.sx, self.sy, self.sx, self.sy, width=2, fill="", dash=(3,5), outline="green3")
        
        self.d1 = self.c.create_line(self.sx, self.sy, self.sx, self.sy, width=1.2, fill="green2")
        self.d2 = self.c.create_line(self.sx, self.sy, self.sx, self.sy, width=1.2, fill="green2")
        
        
    def on_drag(self, event):
        #print("Draged", event)
        self.cx = event.x
        self.cy = event.y

        self.c.coords(self.rect, self.sx, self.sy, self.cx, self.cy)
        self.c.coords(self.d1, self.sx, self.sy, self.cx, self.cy)
        self.c.coords(self.d2, self.cx, self.sy, self.sx, self.cy)
        
        
    def on_release(self, event):
        #print("Released", event)
        self.ex = event.x
        self.ey = event.y
        
        print(self.sx, self.sy, self.ex, self.ey)
        
        self.c.delete(self.rect)
        self.c.delete(self.d1)
        self.c.delete(self.d2)
        self.window.update()
        self.area_screenshot()
        self.close_snip_window()
        
        
    def close_snip_window(self):
        os.remove(self.filename)
        self.window.withdraw()
        self.window.quit()
        
        
    def quitFullScreen(self, event):
        self.fullScreenState = False
        self.window.attributes("-fullscreen", self.fullScreenState)
          

        
s = Snaper()
# fn, x, y = s.get()

# s.get()

# fn, x, y

In [ ]:
# 109 108 274 209

x = (109 + 274) / 2
y = (108 + 209) / 2

hover((x,y,)) 

In [ ]:
# EXAMPLE 1

import tkinter as tk

class Fullscreen_Example:
    def __init__(self):
        self.window = tk.Tk()
        self.window.attributes('-fullscreen', True)  
        self.fullScreenState = False
        self.window.bind("<F11>", self.toggleFullScreen)
        self.window.bind("<Escape>", self.quitFullScreen)

        self.window.mainloop()

    def toggleFullScreen(self, event):
        self.fullScreenState = not self.fullScreenState
        self.window.attributes("-fullscreen", self.fullScreenState)

    def quitFullScreen(self, event):
        self.fullScreenState = False
        self.window.attributes("-fullscreen", self.fullScreenState)

if __name__ == '__main__':
    app = Fullscreen_Example()  

In [ ]:
# EXAMPLE 2 with canvas


from tkinter import *
# import pyautogui

import datetime

class Application():
    def __init__(self, master):
        self.master = master
        self.rect = None
        self.x = self.y = 0
        self.start_x = None
        self.start_y = None
        self.curX = None
        self.curY = None

        # root.configure(background = 'red')
        # root.attributes("-transparentcolor","red")

#         root.attributes("-transparent", "blue")
        root.geometry('400x50+200+200')  # set new geometry
        root.title('Lil Snippy')
        self.menu_frame = Frame(master, bg="blue")
        self.menu_frame.pack(fill=BOTH, expand=YES)

        self.buttonBar = Frame(self.menu_frame,bg="")
        self.buttonBar.pack(fill=BOTH,expand=YES)

        self.snipButton = Button(self.buttonBar, width=3, command=self.createScreenCanvas, background="green")
        self.snipButton.pack(expand=YES)

        self.master_screen = Toplevel(root)
        self.master_screen.withdraw()
#         self.master_screen.attributes("-transparent", "blue")
        self.picture_frame = Frame(self.master_screen, background = "blue")
        self.picture_frame.pack(fill=BOTH, expand=YES)

    def takeBoundedScreenShot(self, x1, y1, x2, y2):
        im = pyautogui.screenshot(region=(x1, y1, x2, y2))
        x = datetime.datetime.now()
        fileName = x.strftime("%f")
        im.save("snips/" + fileName + ".png")

    def createScreenCanvas(self):
        self.master_screen.deiconify()
        root.withdraw()

        self.screenCanvas = Canvas(self.picture_frame, cursor="cross", bg="grey11")
        self.screenCanvas.pack(fill=BOTH, expand=YES)

        self.screenCanvas.bind("<ButtonPress-1>", self.on_button_press)
        self.screenCanvas.bind("<B1-Motion>", self.on_move_press)
        self.screenCanvas.bind("<ButtonRelease-1>", self.on_button_release)

        self.master_screen.attributes('-fullscreen', True)
        self.master_screen.attributes('-alpha', .3)
        self.master_screen.lift()
        self.master_screen.attributes("-topmost", True)

    def on_button_release(self, event):
        self.recPosition()

        if self.start_x <= self.curX and self.start_y <= self.curY:
            print("right down")
            self.takeBoundedScreenShot(self.start_x, self.start_y, self.curX - self.start_x, self.curY - self.start_y)

        elif self.start_x >= self.curX and self.start_y <= self.curY:
            print("left down")
            self.takeBoundedScreenShot(self.curX, self.start_y, self.start_x - self.curX, self.curY - self.start_y)

        elif self.start_x <= self.curX and self.start_y >= self.curY:
            print("right up")
            self.takeBoundedScreenShot(self.start_x, self.curY, self.curX - self.start_x, self.start_y - self.curY)

        elif self.start_x >= self.curX and self.start_y >= self.curY:
            print("left up")
            self.takeBoundedScreenShot(self.curX, self.curY, self.start_x - self.curX, self.start_y - self.curY)

        self.exitScreenshotMode()
        return event

    def exitScreenshotMode(self):
        print("Screenshot mode exited")
        self.screenCanvas.destroy()
        self.master_screen.withdraw()
        root.deiconify()

    def exit_application(self):
        print("Application exit")
        root.quit()

    def on_button_press(self, event):
        # save mouse drag start position
        self.start_x = self.screenCanvas.canvasx(event.x)
        self.start_y = self.screenCanvas.canvasy(event.y)

        self.rect = self.screenCanvas.create_rectangle(self.x, self.y, 1, 1, outline='red', width=3, fill="blue")

    def on_move_press(self, event):
        self.curX, self.curY = (event.x, event.y)
        # expand rectangle as you drag the mouse
        self.screenCanvas.coords(self.rect, self.start_x, self.start_y, self.curX, self.curY)

    def recPosition(self):
        print(self.start_x)
        print(self.start_y)
        print(self.curX)
        print(self.curY)

        
        
        
if __name__ == '__main__':
    root = Tk()
#     root.wait_visibility(root)
    app = Application(root)
    root.mainloop()